In [20]:
import heapq
import numpy as np

def dijkstra(grid, start, goal):
    rows, cols = len(grid), len(grid[0])
    distances = { (i, j): float('inf') for i in range(rows) for j in range(cols) }
    distances[start] = 0
    priority_queue = [(0, start)]
    came_from = {start: None}

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        if current_node == goal:
            break

        for direction in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            neighbor = (current_node[0] + direction[0], current_node[1] + direction[1])
            if 0 <= neighbor[0] < rows and 0 <= neighbor[1] < cols:
                distance = current_distance + grid[neighbor[0]][neighbor[1]]
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    heapq.heappush(priority_queue, (distance, neighbor))
                    came_from[neighbor] = current_node

    path = []
    current = goal
    while current:
        path.append(current)
        current = came_from[current]
    path.reverse()
    return path

grid = [
    [9, 1, 9, 9, 9, 9, 9],
    [9, 1, 1, 9, 1, 1, 9],
    [9, 9, 1, 1, 1, 9, 9],
    [9, 1, 1, 9, 1, 1, 9],
    [9, 9, 9, 9, 1, 9, 9],
    [9, 1, 1, 1, 1, 9, 9],
    [9, 9, 1, 9, 9, 9, 9]
]

start = (0, 1)
goal = (6, 2)

path = dijkstra(grid, start, goal)

def print_grid(grid, path):
    grid_copy = np.array(grid, dtype=object)
    for (r, c) in path:
        grid_copy[r, c] = '°'
    
    grid_copy[grid_copy == 1] = '·'
    grid_copy[grid_copy == 9] = '◊'
    
    for row in grid_copy:
        print(' '.join(str(cell) for cell in row))

print_grid(grid, path)

◊ ° ◊ ◊ ◊ ◊ ◊
◊ ° ° ◊ · · ◊
◊ ◊ ° ° ° ◊ ◊
◊ · · ◊ ° · ◊
◊ ◊ ◊ ◊ ° ◊ ◊
◊ · ° ° ° ◊ ◊
◊ ◊ ° ◊ ◊ ◊ ◊


In [21]:
from IPython.display import Markdown, display

direction_to_words_map = {
    1: 'right',
    4: 'down',
    3: 'left',
    2: 'up'
}

direction_map = {
    (0, 1): 1,
    (1, 0): 4,
    (0, -1): 3,
    (-1, 0): 2
}

def add_directions(path, start_direction):
    path_and_directions = []

    for i in range(1, len(path)):
        prev = path[i - 1]
        curr = path[i]
        move = (curr[0] - prev[0], curr[1] - prev[1])

        path_and_directions.append((prev, direction_map[start_direction], direction_map[move]))
        start_direction = move

    path_and_directions.append((path[-1], direction_map[start_direction], direction_map[start_direction]))

    return path_and_directions

path_and_directions = add_directions(path, (1, 0))

def display_path_and_directions(path_and_directions):
    table = "| Position | Kp | Kg | Start Dir  | End Dir |\n"
    table += "|-|-|-|-|-|\n"
    for i, (position, start_dir, move_dir) in enumerate(path_and_directions):
        table += f"| {position[0]}{position[1]} | {start_dir} | {move_dir} | {direction_to_words_map[int(start_dir)]} | {direction_to_words_map[int(move_dir)]} |\n"
    display(Markdown(table))

display_path_and_directions(path_and_directions)


| Position | Kp | Kg | Start Dir  | End Dir |
|-|-|-|-|-|
| 01 | 4 | 4 | down | down |
| 11 | 4 | 1 | down | right |
| 12 | 1 | 4 | right | down |
| 22 | 4 | 1 | down | right |
| 23 | 1 | 1 | right | right |
| 24 | 1 | 4 | right | down |
| 34 | 4 | 4 | down | down |
| 44 | 4 | 4 | down | down |
| 54 | 4 | 3 | down | left |
| 53 | 3 | 3 | left | left |
| 52 | 3 | 4 | left | down |
| 62 | 4 | 4 | down | down |


In [22]:
move_commands_to_words_map = {
    0: 'move forward',
    1: 'turn right and move',
    3: 'turn left and move'
}

def generate_commands(path_and_directions):
    move_command_mapping = {
        0: 0,
        1: 3,
        2: 2,
        3: 1
    }

    commands = []
    for position, start_dir, move_dir in path_and_directions:
        command = (move_dir - start_dir + 4) % 4
        commands.append((position, move_command_mapping[command]))

    return commands

commands = generate_commands(path_and_directions)

table = "| Position | Command | Named | \n"
table += "|-|-|-|\n"
for position, command in commands:
    table += f"| {position[0]}{position[1]} | {command} | {move_commands_to_words_map[command]} |\n"
display(Markdown(table))

| Position | Command | Named | 
|-|-|-|
| 01 | 0 | move forward |
| 11 | 3 | turn left and move |
| 12 | 1 | turn right and move |
| 22 | 3 | turn left and move |
| 23 | 0 | move forward |
| 24 | 1 | turn right and move |
| 34 | 0 | move forward |
| 44 | 0 | move forward |
| 54 | 1 | turn right and move |
| 53 | 0 | move forward |
| 52 | 3 | turn left and move |
| 62 | 0 | move forward |


In [23]:
direction_map = {
    (0, 1): 'right',
    (1, 0): 'down',
    (0, -1): 'left',
    (-1, 0): 'up'
}

path_and_directions = []

def path_to_directions(path):
    directions = []
    current_direction = (1, 0)

    for i in range(1, len(path)):
        prev = path[i - 1]
        curr = path[i]
        move = (curr[0] - prev[0], curr[1] - prev[1])

        path_and_directions.append((prev, current_direction, direction_map[move]))

        turn_count = 0
        while move != current_direction:
            turn_count += 1
            current_direction = (current_direction[1], -current_direction[0])

        if turn_count == 1:
            directions.append(move_commands[1])
        elif turn_count == 2:
            directions.append(move_commands[1])
            directions.append(move_commands[1])
        elif turn_count == 3:
            directions.append(move_commands[2])
        
        directions.append(move_commands[0])

    return directions

directions = path_to_directions(path)
for direction in directions:
    print(direction)

NameError: name 'move_commands' is not defined

In [ ]:
sensor_commands = [
    'look forward',
    'look right',
    'look left',
]

sensor_directions = []

for i, direction in enumerate(directions):
    next_command = sensor_commands[0]
    for j in range(i, len(directions)):
        if j != move_commands[0]:
            if directions[j] == move_commands[1]:
                next_command = sensor_commands[1]
                break
            elif directions[j] == move_commands[2]:
                next_command = sensor_commands[2]
                break
    sensor_directions.append(next_command)

for i, direction in enumerate(directions):
    print(f'{direction} -> {sensor_directions[i]}')

0 -> look left
2 -> look left
0 -> look right
1 -> look right
0 -> look left
2 -> look left
0 -> look right
0 -> look right
1 -> look right
0 -> look right
0 -> look right
0 -> look right
1 -> look right
0 -> look left
0 -> look left
2 -> look left
0 -> look forward
